<a href="https://colab.research.google.com/github/TheDukato/Matematyka_w_projektowaniu_sieci/blob/main/Piaskownica_notatnika_PROJEKT_MATEMATYKA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Podział prac:
*   Jakub Sośniak: implementacja modelu
*   Kamil Pokusa: lider, pomoc w implementacji
*   Norbert Barczyk: rozpisanie ograniczeń w LateX, pomoc w impementacji
*   Filip Pieniążek: implementacja modelu,





In [ ]:
try:
  import gdown
except ModuleNotFoundError:
  !pip install gdown

FILE_ID = '1F25HJNcYXxbLjlbdMhKUnq_T0p5BA5A9'
gdown.download(id=FILE_ID)
from mpsis_setup import mpsis
mpsis.set_software()
import pulp

In [2]:
#@title Biblioteki

import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
url = "https://drive.google.com/file/d/1SDAB8yrrvI3JatLSQizzT_3oHlQVEExk/view?usp=sharing"
gdown.download(url=url, quiet=False, fuzzy=True)

# plik https://drive.google.com/file/d/1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf/view?usp=sharing
#FILE_ID = '1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf'
#gdown.download(id=FILE_ID)

from mpsis_setup import mpsis
mpsis.set_software()

import pulp as plp

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1SDAB8yrrvI3JatLSQizzT_3oHlQVEExk 



Mounted at /content/drive


Temat:LP-WSC: a linear programming approach for web service
composition in geographically distributed cloud
environments

*   **Funkcja celu:**

$$ \max \, Z = ( \frac{1}{\sum_{v \in V} I_v \times WScost(v)} + \frac{1}{\sum_{v \in V} I_v \times WS\_Resp(v)} + \sum_{v \in V} I_v \times WS\_Avail(v)+\sum_{v \in V} I_v \times WS\_Reli(v)) $$


* **Ograniczenia:**

\begin{align*}
I_v \times WS_{\text{Cost}}(v) &\leq SLA_v \cdot SL01 \\
I_v \times WS_{\text{Resp}}(v) &\leq SLA_v \cdot SL02 \\
I_v \times WS_{\text{Avail}}(v) &\leq SLA_v \cdot SL03 \\
I_v \times WS_{\text{Reli}}(v) &\leq SLA_v \cdot SL04 \\
I_v &= 0 \text{ or } 1, \forall v \in [0, 1] \\
\text{Count}(I) &\leq \text{Req_Service_Count}
\end{align*}

Z faktu że w wybranym artykule występują nieliniowości musimy przeprowadzić linearyzację dwóch składników sumy, wynikją z tego dodatkowe ograniczenia i funkcja celu przyjmuje inną postać:
*   **Indeksy**
$$
v - \text{wezel  sieci}
$$
*   **Stale**
$$
SLA,SLO - \text{paramtery związane z QoS dla danego uzytkownika}
$$
*   **Funkcja celu:**

$$
  \max \, Z = (
  \sum_{v \in V} z_v \times WS_{\text{cost}}(v) +
  \sum_{v \in V} y_v \times WS_{\text{_Resp}}(v) +
  \sum_{v \in V} I_v \times WS\_Avail(v) +
  \sum_{v \in V} I_v \times WS\_Reli(v))
$$


* **Ograniczenia:**

\begin{align*}
I_v \times WS_{\text{Cost}}(v) &\leq SLA_v \cdot SL01 \\
I_v \times WS_{\text{Resp}}(v) &\leq SLA_v \cdot SL02 \\
I_v \times WS_{\text{Avail}}(v) &\leq SLA_v \cdot SL03 \\
I_v \times WS_{\text{Reli}}(v) &\leq SLA_v \cdot SL04 \\
z_v \geq l_1 + I_v -1 \\
y_v \geq l_2 + I_v -1 \\
\sum_{v} z_v = 1\\
\sum_{v} y_v = 1\\
I_v &= 0 \text{ or } 1, \forall v \in [0, 1] \\
\text{Count}(I) &\leq \text{Req_Service_Count}
\end{align*}


In [ ]:
#@title Dane do modelu

serwer_list= [
    {"cost": 3, "Reliability": 99.95, "Availability": 98, "Response_Time": 9},
    {"cost": 2.5, "Reliability": 90, "Availability": 90, "Response_Time": 12},
    {"cost": 1, "Reliability": 85, "Availability": 85, "Response_Time": 20},
    {"cost": 0.9, "Reliability": 85, "Availability": 90, "Response_Time": 25}
]


SLA_List = {
    "SLA1": {
        "SLO1": 0.5,
        "SLO2": 99.95,
        "SLO3": 99.94,
        "SLO4": 0.125,
    },
    "SLA2": {
        "SLO1": 0.7,
        "SLO2": 99.85,
        "SLO3": 99.99,
        "SLO4": 0.207,
    },
}
"""
    "SLA2": {
        "SLO1": 0.7,
        "SLO2": 99.85,
        "SLO3": 99.99,
        "SLO4": 0.207,
    },
"""
print(serwer_list[0])
print(serwer_list[0]["cost"])
print(serwer_list[0]["Reliability"])
print(serwer_list[0]["Availability"])
print(serwer_list[0]["Response_Time"])
print(SLA_List["SLA1"])
print(SLA_List["SLA1"]["SLO1"])
#print(SLA_List["SLA2"])
#print(SLA_List["SLA2"]["SLO3"])

Req_Service_Count = 1000

{'cost': 3, 'Reliability': 99.95, 'Availability': 98, 'Response_Time': 9}
3
99.95
98
9
{'SLO1': 0.5, 'SLO2': 99.95, 'SLO3': 99.94, 'SLO4': 0.125}
0.5


In [3]:
#@title Generator do danych serwerów
serwer_list=[]
NUMBER_OF_SERWER = 3
for x in range(NUMBER_OF_SERWER):
  serwer_list.append({
      "cost": round(random.uniform(0.5,4),2),
      "Reliability": round(random.uniform(0.9,1),2),
      "Availability": round(random.uniform(0.95,1),2),
      "Response_Time": round(random.uniform(100,3000),2)
      }
  )
for x in range(NUMBER_OF_SERWER):
  print(f"{serwer_list[x]}")

{'cost': 0.56, 'Reliability': 0.95, 'Availability': 0.99, 'Response_Time': 247.07}
{'cost': 3.78, 'Reliability': 0.94, 'Availability': 0.99, 'Response_Time': 1773.25}
{'cost': 2.1, 'Reliability': 0.98, 'Availability': 0.97, 'Response_Time': 2248.76}


In [4]:
#@title Generator do danych SLA

SLA_List = {
    "SLA1": {
        "SLO1": round(random.uniform(0.5,2.0),2),
        "SLO2": round(random.uniform(0.5,2.0),2),
        "SLO3": round(random.uniform(0.5,2.0),2),
        "SLO4": round(random.uniform(0.5,2.0),2),
    },
    "SLA2": {
        "SLO1": round(random.uniform(0.5,2.0),2),
        "SLO2": round(random.uniform(0.5,2.0),2),
        "SLO3": round(random.uniform(0.5,2.0),2),
        "SLO4": round(random.uniform(0.5,2.0),2),
    },
}
NUMBER_OF_SLA=1
SLA_List={}
for x in range(NUMBER_OF_SLA):
  SLA_List[f"SLA{x}"]={
        "SLO1": round(random.uniform(0.5,2.0),2),
        "SLO2": round(random.uniform(0.5,2.0),2),
        "SLO3": round(random.uniform(0.5,2.0),2),
        "SLO4": round(random.uniform(0.5,2.0),2),
  }
for key,value in SLA_List.items():
  print(f"{key}: {value}\n")

SLA0: {'SLO1': 1.1, 'SLO2': 1.15, 'SLO3': 1.35, 'SLO4': 1.13}



In [5]:
#@title Liczba żądań
list_Req_Service_Count = [1000,2000,3000]

In [ ]:
def model_1(llist_Req_Service_Count):
 for Req_Service_Count in llist_Req_Service_Count:
  V = range(len(serwer_list))  # Wezły sieci

  # Inicjalizacja problemu
  prob = pulp.LpProblem(name="QoS_Optimization", sense=pulp.LpMaximize)

  # Zmienne decyzyjne
  I = {v: pulp.LpVariable(name=f"I_{v}", cat="Binary") for v in V}
  z_c = {v: pulp.LpVariable(name=f"z_c_{v}", cat="Continuous") for v in V}
  z_t = {v: pulp.LpVariable(name=f"z_t_{v}", cat="Continuous") for v in V}

  # Funkcja celu
  prob += pulp.lpSum(z_c[v] + z_t[v] + I[v] * serwer_list[v]["Availability"] + I[v] * serwer_list[v]["Reliability"] for v in V)

  # Ograniczenia SLA
  for sla, slo_values in SLA_List.items():
      for v in V:
          prob += I[v] * serwer_list[v]["cost"] <= slo_values["SLO1"]
          prob += I[v] * serwer_list[v]["Response_Time"] <= slo_values["SLO2"]
          prob += I[v] * serwer_list[v]["Availability"] <= slo_values["SLO3"]
          prob += I[v] * serwer_list[v]["Reliability"] <= slo_values["SLO4"]

  # Dodatkowe ograniczenia
  for v in V:
      prob += z_c[v] >= I[v] * serwer_list[v]["cost"] -1
      prob += z_t[v] >= I[v] * serwer_list[v]["Response_Time"] -1

  prob += pulp.lpSum(z_c[v] for v in V) == 1
  prob += pulp.lpSum(z_t[v] for v in V) == 1
  prob += pulp.lpSum(I[v] for v in V) <= Req_Service_Count

  # Model
  print(f"prob {prob}")


  # Rozwiązanie problemu
  prob.solve()

  # Wyświetlenie wyniku
  print("Status:", plp.LpStatus[prob.status])
  print("Wartość funkcji celu (Z):", round(plp.value(prob.objective), 2))


  #print("Rozwiązanie: ", plp.LpStatus[model.status])
  #print("Liczba zmiennych: " + str(model.numVariables()))
  #print("Liczba ograniczeń: " + str(model.numConstraints()))
  #print("Całkowity koszt: " + str(plp.value(model.objective)))
  #for x_dp in x:
  #  if x[x_dp].varValue != 0:
  #    print("Wielkosc przepływu dla zapotrzebowania " + str(x_dp[0]) + " dla sciezki nr " + str(x_dp[1]) + " wynosi: " + str(x[x_dp].varValue))


  # Wyświetlenie wartości zmiennych decyzyjnych
  for v in V:
      print(f"I[{v}] =", int(I[v].value()))
      #print(f"z[{v}] =", int(z[v].value()))
      #print(f"y[{v}] =", int(y[v].value()))
model_1(list_Req_Service_Count)

In [ ]:
def model_2(list_Req_Service_Count):
 for Req_Service_Count in list_Req_Service_Count:
    V = range(len(serwer_list))  # Wezły sieci

    # Inicjalizacja problemu
    prob = pulp.LpProblem(name="QoS_Optimization", sense=pulp.LpMaximize)

    # Zmienne decyzyjne
    I = {v: pulp.LpVariable(name=f"I_{v}", cat="Binary") for v in V}

    # Funkcje celu
    C_total = pulp.lpSum(I[v] * serwer_list[v]["cost"] for v in V)
    T_total = pulp.lpSum(I[v] * serwer_list[v]["Response_Time"] for v in V)
    A_total = pulp.lpSum(I[v] * serwer_list[v]["Availability"] for v in V)
    R_total = pulp.lpSum(I[v] * serwer_list[v]["Reliability"] for v in V)

    prob += - C_total - T_total + A_total + R_total


    # Ograniczenia SLA
    for sla, slo_values in SLA_List.items():
        for v in V:
            prob += I[v] * serwer_list[v]["cost"] <= slo_values["SLO1"]
            prob += I[v] * serwer_list[v]["Response_Time"] <= slo_values["SLO2"]
            prob += I[v] * serwer_list[v]["Availability"] <= slo_values["SLO3"]
            prob += I[v] * serwer_list[v]["Reliability"] <= slo_values["SLO4"]

    # Dodatkowe ograniczenia
    prob += T_total == pulp.lpSum(I[v] * serwer_list[v]["Response_Time"] for v in V)
    prob += A_total == pulp.lpSum(I[v] * serwer_list[v]["Availability"] for v in V)
    prob += R_total == pulp.lpSum(I[v] * serwer_list[v]["Reliability"] for v in V)

    for v in V:
        prob += I[v] == 0 or I[v] == 1

    prob += pulp.lpSum(I[v] for v in V) <= Req_Service_Count

    # Rozwiązanie problemu
    prob.solve()


    # Wyświetlenie wyniku
    print("Status:", plp.LpStatus[prob.status])
    print("Wartość funkcji celu (Z):", round(plp.value(prob.objective), 2))

    # Wyświetlenie wartości zmiennych decyzyjnych
    for v in V:
      print(f"I[{v}] =", int(I[v].value()))
      #print(f"z[{v}] =", int(z[v].value()))
      #print(f"y[{v}] =", int(y[v].value()))
      #print(f"l1[{v}] =", round(l1[v].value(), 2))
      #print(f"l2[{v}] =", round(l2[v].value(), 2))
model_2(list_Req_Service_Count)

Status: Optimal
Wartość funkcji celu (Z): 0.0
I[0] = 0
I[1] = 0
I[2] = 0
Status: Optimal
Wartość funkcji celu (Z): 0.0
I[0] = 0
I[1] = 0
I[2] = 0
Status: Optimal
Wartość funkcji celu (Z): 0.0
I[0] = 0
I[1] = 0
I[2] = 0


\begin{align*}
&y \geq \frac{1}{\sum_{v \in V} I_v \times \text{WScost}(v)} \quad (1) \\
&y \cdot \sum_{v \in V} I_v \times WScost(v) \geq 1 \quad \text{(2)} \\
&y \leq M \cdot \sum_{v \in V} I_v \times WScost(v) \quad \text{(3)} \\
&\sum_{v \in V} I_v \times WScost(v) \leq M \cdot (1 - I_v) \quad \text{(4)}
\end{align*}

\\
dupa


\begin{align*}
&z \geq \frac{1}{\sum_{v \in V} I_v \times \text{WS\_Resp}(v)} \quad (5) \\
&z \cdot \sum_{v \in V} I_v \times WS_Resp(v) \geq 1 \quad \text{(6)} \\
&z \leq N \cdot \sum_{v \in V} I_v \times WS_Resp(v) \quad \text{(7)} \\
&\sum_{v \in V} I_v \times WS_Resp(v) \leq N \cdot (1 - I_v) \quad \text{(8)}
\end{align*}

In [12]:
def model3(llist_Req_Service_Count):
  for Req_Service_Count in llist_Req_Service_Count:
    # Utwórz problem liniowy
    model_3 = None
    model_3 = plp.LpProblem(name="Maximize_Z", sense=plp.LpMaximize)

    V = range(len(serwer_list))

    # Zmienne decyzyjne
    I = {v: plp.LpVariable(name=f"I_{v}", cat="Binary") for v in V}
    D = {v: plp.LpVariable(name=f"D_{v}",lowBound = 0, cat="Continuous") for v in V}
    W = {v: plp.LpVariable(name=f"W_{v}",lowBound = 0, cat="Continuous") for v in V}
    g = {v: plp.LpVariable(name=f"g_{v}",lowBound = 0, cat="Continuous") for v in V}
    z = {v: plp.LpVariable(name=f"z_{v}",lowBound = 0, cat="Continuous") for v in V}

    # Funkcja celu zlinearyzowana
    A_total = plp.lpSum(I[v] * serwer_list[v]["Availability"] for v in V)
    R_total = plp.lpSum(I[v] * serwer_list[v]["Reliability"] for v in V)

    model_3 += plp.lpSum(W[v] + D[v] for v in V) + A_total + R_total


    # Ograniczenia
    for sla, slo_values in SLA_List.items():
      for v in V:
        model_3 += I[v] * serwer_list[v]["cost"] <= slo_values["SLO1"]
        model_3 += I[v] * serwer_list[v]["Response_Time"] <= slo_values["SLO2"]
        model_3 += I[v] * serwer_list[v]["Availability"] <= slo_values["SLO3"]
        model_3 += I[v] * serwer_list[v]["Reliability"] <= slo_values["SLO4"]

    # Zlinearyzowane ograniczenia dla D
    model_3 += plp.lpSum( z[v] * serwer_list[v]["cost"] for v in V) == 1
    for v in V:
      model_3 += plp.lpSum( z[v] >= W[v] + I[v] -1 )

    # Zlinearyzowane ograniczenia dla z
    model_3 += plp.lpSum( g[v] * serwer_list[v]["Response_Time"] for v in V) == 1
    for v in V:
      model_3 += plp.lpSum( g[v] >= D[v] + I[v] -1)

    # Ograniczenie dotyczące liczby wymaganych usług
    model_3 += plp.lpSum(I[v] for v in V) <= Req_Service_Count

    #print(model_3)

    # Rozwiązanie problemu
    model_3.solve()

    # Wyświetlenie wyników
    print(f"Status: {model_3.status}, Koszt: {model_3.objective.value()}")
    for v in V:
        print(f"Server {v+1}: {I[v].value()}")
model3(list_Req_Service_Count)

Status: 1, Koszt: 1.00044468952
Server 1: 0.0
Server 2: 0.0
Server 3: 0.0
Status: 1, Koszt: 1.00044468952
Server 1: 0.0
Server 2: 0.0
Server 3: 0.0
Status: 1, Koszt: 1.00044468952
Server 1: 0.0
Server 2: 0.0
Server 3: 0.0
